# **Week7. Framework assignment (pytorch)**
13기 이유민

---

## **신경망 학습 과정**

1. 신경망 정의
2. 데이터셋 입력 (반복해서!)
3. 입력된 데이터셋을 신경망에서 전파
4. loss 계산 (출력된 값과 정답의 차이)
5. 그라디언트를 역전파
6. 가중치 갱신
   
   규칙: weight = weight - learning_rate * gradient

+) torch.nn 패키지는 autograd로 모델을 정의& 미분함.

backward는 autograd를 통해 자동으로 정의되기 때문에 따로 정의하지 않음


  ---

## **기본 설정**

In [1]:
cd/content/drive/My Drive/tobigs

/content/drive/My Drive/tobigs


In [2]:
ls

8.67torch_submission.csv     radam_submission3.csv  train_df.csv
8.67torch_submission.gsheet  radam_submission.csv   train_df.csv.zip
kaggle.json                  sample_submission.csv  Untitled0.ipynb
keras_sample.csv             test_df.csv            Week7_Keras.ipynb
radam_submission1.csv        test_df.csv.zip        week7_pytorch.ipynb
radam_submission2.csv        torch_submission.csv


## **데이터 전처리**

In [0]:
import pandas as pd

sample_submission = pd.read_csv("sample_submission.csv")
train = pd.read_csv("train_df.csv")
test = pd.read_csv("test_df.csv")

In [0]:
X = train.iloc[:,1:].values / 255
X_test = test.iloc[:,1:].values / 255 # 픽셀값인 255로 나눠줍니당

y = train.iloc[:,0].values

## **Dataset 정의**

In [0]:
from torch.utils.data import Dataset, DataLoader

class TrainDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return X, y

In [0]:
class TestDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        return X

## **Loader 정의**

In [0]:
traindataloader = DataLoader(TrainDataset(X, y), batch_size=128, shuffle=True, num_workers=4) 
 # 배치 사이즈만큼 데이터 가져 옴!
# train 은 섞어 주는 게 적절함 (test는 섞지 않음!))
testdataloader = DataLoader(TestDataset(X_test, y=None), batch_size=4, shuffle=False, num_workers=4) 

## **신경망 정의**

+) 이번 과제에서 사용하지는 않았지만, auto incoder는 convolution 연산을 통해 이미지를 압축함.

  보통 2의 배수로 사이즈를 정해 주는 경우가 많은데, 이는 원래 사이즈로 복원할 때 2의 배수가 효율적이기 때문!


+) 출력할 때 softmax를 적용시키지 않는 건 nn.CrossEntropy Loss가 소프트맥스를 통과시킨 y와 Loss를 알아서 계산해 주기 때문임!

In [8]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class Net(nn.Module):

    # 데이터셋 만들 때 들어가는 인자들
    def __init__(self): 
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512) # mnist는 784 
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 256)
        self.fc4 = nn.Linear(256, 10)
        self.elu = nn.ELU()
        self.dropout1 = torch.nn.Dropout(p=0.5)
        self.dropout2 = torch.nn.Dropout(p=0.5)

    # Activation function
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(F.relu(x))
        x = self.dropout1(x)   # dropout을 통해 regulation
        x = self.fc3(self.elu(x))
        x = self.dropout2(x)
        x = self.fc4(x)
        return x


net = Net() # dictionary 형태
print(net) # 확인!

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=10, bias=True)
  (elu): ELU(alpha=1.0)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)


torch.optim 패키지를 통해 갱신 규칙 구현 가능

In [0]:
import torch.optim as optim # optimizer 함수들

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001, weight_decay=1e-3)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [11]:
net.to(device)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=10, bias=True)
  (elu): ELU(alpha=1.0)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)

In [12]:
num_epochs = 30
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(traindataloader):
        inputs, labels = data
        inputs = torch.tensor(inputs, device=device).float()
        labels = torch.tensor(labels, device=device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # print(type(inputs), type(labels), type(outputs))
        # forward + backward + optimize
        outputs = net(inputs)
        # print(type(inputs), type(labels), type(outputs))
        
        loss = criterion(outputs, labels)
         # 여기서 loss는 crossentropy로 계산
        loss.backward()
        # backward에서 backpropagation 을 통해 gradient 계산

        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print("Epoch : {} loss: {}".format(epoch, running_loss))
print('Finished Training')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Epoch : 0 loss: 313.64913798868656
Epoch : 1 loss: 74.48248929530382
Epoch : 2 loss: 56.81258722394705
Epoch : 3 loss: 48.90183378756046
Epoch : 4 loss: 43.11422214657068
Epoch : 5 loss: 38.83926794305444
Epoch : 6 loss: 35.39365341514349
Epoch : 7 loss: 32.26212779805064
Epoch : 8 loss: 29.46916787698865
Epoch : 9 loss: 26.843082256615162
Epoch : 10 loss: 25.115593686699867
Epoch : 11 loss: 22.739564958959818
Epoch : 12 loss: 21.206433007493615
Epoch : 13 loss: 19.35347837768495
Epoch : 14 loss: 17.964565010741353
Epoch : 15 loss: 16.964374106377363
Epoch : 16 loss: 16.217251947149634
Epoch : 17 loss: 14.606696631759405
Epoch : 18 loss: 13.845450356602669
Epoch : 19 loss: 13.066611418500543
Epoch : 20 loss: 12.311219248920679
Epoch : 21 loss: 11.59735669568181
Epoch : 22 loss: 10.879574364051223
Epoch : 23 loss: 10.846230681985617
Epoch : 24 loss: 10.128522692248225
Epoch : 25 loss: 9.718724505975842
Epoch : 26 loss: 9.33764842711389
Epoch : 27 loss: 8.986123587936163
Epoch : 28 loss:

In [13]:
correct = 0
total = 0
net.eval()
preds = []
with torch.no_grad():
    for data in testdataloader:
        inputs = data
        inputs = torch.tensor(inputs, device=device).float()
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        for pred in predicted:
          preds.append(pred.cpu().numpy())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [14]:
preds[:10]

[array(8),
 array(0),
 array(5),
 array(3),
 array(8),
 array(1),
 array(9),
 array(6),
 array(6),
 array(0)]

In [15]:
import numpy as np
preds = np.array(preds)
print(preds.shape)

(18000,)


In [16]:
sample_submission["Category"] = pd.Series(preds)
sample_submission.head(20)

,Id,Category
0,57808,8
1,4960,0
2,35755,5
3,15543,3
4,48968,8
5,55461,1
6,19499,9
7,49696,6
8,40176,6
9,39480,0


In [0]:
# 제출 파일 저장
sample_submission.to_csv("torch_submission.csv", index=False)